In [1]:
!sudo service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
/opt/conda/bin/xz
   ...done.


In [2]:
# Example -  connecting to the database "mydatabase":

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="admin",
  database="Northwind_DB" # Make sure the database is selected
)

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('Northwind_DB',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('test',)
('w3schools_database',)


In [6]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

def execute_query(query, params=None, many=False):
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="admin",
            database="Northwind_DB"
        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            if many:
                cursor.executemany(query, params)
            else:
                cursor.execute(query, params)
            
            # If the query is a SELECT statement, fetch and return the results as a DataFrame
            if query.strip().upper().startswith("SELECT"):
                results = cursor.fetchall()
                df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
                for row in results:
                    #print(row)
                    pass
                return df

                display(df)
            else:
                # Commit the transaction for INSERT, UPDATE, DELETE queries
                connection.commit()
                return cursor.rowcount
        
    except Error as e:
        print(f"Error: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Create the Products table
create_table_query = """
CREATE TABLE IF NOT EXISTS Products (
    ProductID INT,
    ProductName VARCHAR(255),
    SupplierID INT,
    CategoryID INT,
    Unit VARCHAR(255),
    Price DECIMAL(10, 2)
)
"""
execute_query(create_table_query)

# Insert data into the Products table
insert_data_query = """
INSERT INTO Products (ProductID, ProductName, SupplierID, CategoryID, Unit, Price) VALUES (%s, %s, %s, %s, %s, %s)
"""
products_data = [
    (1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.00),
    (2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.00),
    (3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.00),
    (4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.00),
    (5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35)
]
execute_query(insert_data_query, products_data, many=True)

print("Products table created and data inserted successfully.")


Products table created and data inserted successfully.


In [5]:
select_query = "DROP TABLE Products;"
execute_query(select_query)

0

SELECT * FROM Products
ORDER BY Price DESC;

In [7]:
select_query = "SELECT * FROM Products ORDER BY Price DESC;"
execute_query(select_query)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
1,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
2,2,Chang,1,1,24 - 12 oz bottles,19.00
3,1,Chais,1,1,10 boxes x 20 bags,18.00
4,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00


In [8]:
# Query to list tables
list_tables_query = """
SELECT *
FROM information_schema.tables
WHERE table_schema = 'Northwind_DB'
ORDER BY table_name;
"""
df_tables = execute_query(list_tables_query)
df_tables

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE,ENGINE,VERSION,ROW_FORMAT,TABLE_ROWS,AVG_ROW_LENGTH,DATA_LENGTH,...,INDEX_LENGTH,DATA_FREE,AUTO_INCREMENT,CREATE_TIME,UPDATE_TIME,CHECK_TIME,TABLE_COLLATION,CHECKSUM,CREATE_OPTIONS,TABLE_COMMENT
0,def,Northwind_DB,CustomerView,VIEW,None,NaN,None,NaN,NaN,NaN,...,NaN,NaN,None,2024-11-06 14:58:38,NaT,None,None,None,None,VIEW
1,def,Northwind_DB,Customers,BASE TABLE,InnoDB,10.0,Dynamic,12.0,1365.0,16384.0,...,0.0,0.0,None,2024-11-05 14:32:49,NaT,None,utf8mb4_0900_ai_ci,None,,
2,def,Northwind_DB,Products,BASE TABLE,InnoDB,10.0,Dynamic,5.0,3276.0,16384.0,...,0.0,0.0,None,2024-11-06 16:01:53,2024-11-06 16:01:53,None,utf8mb4_0900_ai_ci,None,,
3,def,Northwind_DB,Sales,BASE TABLE,InnoDB,10.0,Dynamic,5.0,3276.0,16384.0,...,0.0,0.0,None,2024-11-06 14:15:48,NaT,None,utf8mb4_0900_ai_ci,None,,


In [10]:
# List Routines

query = """
SELECT  ROUTINE_SCHEMA, ROUTINE_NAME, ROUTINE_TYPE
FROM INFORMATION_SCHEMA.ROUTINES
WHERE ROUTINE_SCHEMA = 'Northwind_DB';
"""
execute_query(query)

,ROUTINE_SCHEMA,ROUTINE_NAME,ROUTINE_TYPE
0,Northwind_DB,GetCustomerCount,FUNCTION
1,Northwind_DB,AddCustomer,PROCEDURE


In [87]:
query = "SHOW ENGINE INNODB STATUS;" 

#df_result = execute_query(query)
#display(df_result)

In [12]:
query = """
SELECT * FROM Products;
"""
execute_query(query)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35


```SQL
SELECT MIN(Price) 
FROM Products;

// Syntax
SELECT MIN(column_name)
FROM table_name
WHERE condition;
```

In [11]:
query = """
SELECT MIN(Price) FROM Products;
"""
execute_query(query)

,MIN(Price)
0,10.00


In [14]:
query = """
SELECT MIN(Price) as Min_Price FROM Products;
"""
execute_query(query)

,Min_Price
0,10.00


In [15]:
query = """
SELECT MAX(Price) as Max_Price FROM Products;
"""
execute_query(query)

,Max_Price
0,22.00


Use `MIN() with GROUP BY`
Here we use the MIN() function and the GROUP BY clause, to return the smallest price for each category in the Products table:

```SQL
SELECT MIN(Price) AS SmallestPrice, CategoryID
FROM Products
GROUP BY CategoryID;
```

In [20]:
query = """
SELECT MIN(Price) AS SmallestPrice, CategoryID
FROM Products
GROUP BY CategoryID;
"""
execute_query(query)

,SmallestPrice,CategoryID
0,18.00,1
1,10.00,2


In [21]:
query = """
SELECT MIN(Price) AS SmallestPrice
FROM Products
GROUP BY CategoryID;
"""
execute_query(query)

,SmallestPrice
0,18.00
1,10.00


```SQL
SELECT COUNT(*)
FROM Products;

// Syntax
SELECT COUNT(column_name)
FROM table_name
WHERE condition;
```

In [23]:
query = """
SELECT COUNT(*) FROM Products;
"""
execute_query(query)

,COUNT(*)
0,5


In [ ]:
# Find the number of products where the ProductName is not null:

SELECT COUNT(ProductName)
FROM Products;